In [33]:
import numpy as np
import pandas as pd
from data_loader import load_names_from_web, births_in_year
import altair as alt
alt.data_transformers.disable_max_rows()

ImportError: cannot import name 'births_in_year' from 'data_loader' (/Users/194627/Sites/capstone/data_loader.py)

In [29]:
dfraw = load_names_from_web(category='national', hide_pre_1937=False, use_existing_files=True)

first_known_year = dfraw['year'].min()
last_known_year = dfraw['year'].max()

# dfraw = dfraw[dfraw['name'] == 'Sarah']

all_years = dfraw.copy()

In [30]:
data = pd.DataFrame()

for year in range(first_known_year, last_known_year+1):

    df = all_years.copy()
    df = df[df['year'] <= year]
    df = df.sort_values(by='year')
    df['cumsum'] = df.groupby(['name', 'M/F'])['count'].cumsum()
    df['sum'] = df.groupby(['name', 'M/F'])['count'].transform('sum')
    medians = df[df['cumsum'] >= df['sum']/2]
    medians = medians.drop_duplicates(subset=['name', 'M/F'], keep='first')
    medians['age'] = year - medians['year']
    thisyear = df[df['year'] == year][['name', 'M/F', 'count']].rename(columns={'count': 'thisyear_count'})
    df = medians.merge(thisyear, how='left', on=['name', 'M/F'])
    df['thisyear_count'] = df['thisyear_count'].fillna(0)
    df['thisyear_rank'] = df['thisyear_count'].rank(method='first', ascending=False)
    df = df[['state', 'name', 'M/F', 'sum', 'age', 'thisyear_count', 'thisyear_rank']]
    df['year'] = year
    # display(df)
    data = pd.concat([data, df])

data

,state,name,M/F,sum,age,thisyear_count,thisyear_rank,year
0,US,Mary,F,7065,0,7065.0,3.0,1880
1,US,Vern,M,19,0,19.0,766.0,1880
2,US,Stewart,M,19,0,19.0,767.0,1880
3,US,Randolph,M,19,0,19.0,768.0,1880
4,US,Lucien,M,19,0,19.0,769.0,1880
...,...,...,...,...,...,...,...,...
113877,US,Damiri,M,46,0,27.0,9065.0,2022
113878,US,Kyliam,M,56,0,35.0,7564.0,2022
113879,US,Jahkarri,M,36,0,36.0,7407.0,2022
113880,US,Asta,M,63,0,32.0,8035.0,2022


In [31]:
data = data[data['thisyear_rank'] <= 100]
data_F = data[data['M/F'] == 'F']
data_M = data[data['M/F'] == 'M']

In [32]:
chart_M = alt.Chart(data_M).mark_line().encode(
    x='thisyear_rank',
    y='thisyear_count',
    tooltip=['name', 'M/F', 'year', 'age', 'thisyear_count', 'sum'],
    color='year'
).properties(
    width=450,
    height=400
)
chart_F = alt.Chart(data_F).mark_line().encode(
    x='thisyear_rank',
    y='thisyear_count',
    tooltip=['name', 'M/F', 'year', 'age', 'thisyear_count', 'sum'],
    color='year'
).properties(
    width=450,
    height=400
)
chart_M | chart_F

# ranks need to be cleaned up by gender

alt.HConcatChart(...)

In [ ]:
births_in_year(1937)